In [7]:
import os
import sys
sys.path.append(os.path.abspath(os.getcwd()))

import pandas as pd
import sqlite3
import pickle
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import yaml
import csv
import re

print("Imports successful")

Imports successful


In [2]:
def load_config(config_path="config/config.yaml"):
    with open(config_path, "r") as f:
        return yaml.safe_load(f)

def is_valid_recommendation(rec):
    if not rec or not isinstance(rec, str):
        return False
    pattern = r"^\d+\..*\n.*\d+\..*$"
    return bool(re.match(pattern, rec, re.MULTILINE)) and 4 <= len(rec.split("\n")) <= 6

def extract_aspect(comment):
    if "refund" in comment.lower():
        return "refund speed"
    elif "app" in comment.lower() or "crash" in comment.lower():
        return "app stability"
    elif "support" in comment.lower():
        return "support quality"
    else:
        return "general"

def get_prewritten_recommendations(aspect):
    recommendations = {
        "refund speed": """1. Reduce refund processing time to 5 business days.
2. Implement automated refund tracking system.
3. Train staff on efficient refund procedures.
4. Provide real-time refund status updates to customers.""",
        "app stability": """1. Conduct thorough app testing before updates.
2. Fix known crash issues in the next release.
3. Optimize app performance for low-end devices.
4. Offer users a feedback channel for crash reports.""",
        "support quality": """1. Train support team on empathy and problem-solving.
2. Reduce support ticket response time to 24 hours.
3. Implement a 24/7 live chat support option.
4. Create a detailed FAQ for common issues.""",
        "general": """1. Improve overall customer communication.
2. Gather regular feedback through surveys.
3. Enhance user interface for better experience.
4. Offer loyalty discounts to retain customers."""
    }
    return recommendations.get(aspect, recommendations["general"])

print("Helper functions defined")

Helper functions defined


In [3]:
# database.py logic
def init_db(db_path="data/feedback.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS feedback (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            brand TEXT,
            comment TEXT,
            tone TEXT,
            embedding BLOB,
            recommendations TEXT,
            aspect TEXT
        )
    """)
    cursor.execute("PRAGMA table_info(feedback)")
    columns = [col[1] for col in cursor.fetchall()]
    if "embedding" not in columns:
        cursor.execute("ALTER TABLE feedback ADD COLUMN embedding BLOB")
    if "recommendations" not in columns:
        cursor.execute("ALTER TABLE feedback ADD COLUMN recommendations TEXT")
    if "aspect" not in columns:
        cursor.execute("ALTER TABLE feedback ADD COLUMN aspect TEXT")
    conn.commit()
    conn.close()

init_db()
conn = sqlite3.connect("data/feedback.db")
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(feedback)")
print("Feedback table schema:")
print(cursor.fetchall())
conn.close()

Feedback table schema:
[(0, 'id', 'INTEGER', 0, None, 1), (1, 'brand', 'TEXT', 0, None, 0), (2, 'comment', 'TEXT', 0, None, 0), (3, 'tone', 'TEXT', 0, None, 0), (4, 'embedding', 'BLOB', 0, None, 0), (5, 'recommendations', 'TEXT', 0, None, 0), (6, 'aspect', 'TEXT', 0, None, 0)]


In [4]:
def populate_db():
    config = load_config()
    db_path = config["database"]["path"]
    embedder_model = config["models"]["embedder"]
    process_neutral = config.get("processing", {}).get("include_neutral", False)

    init_db(db_path)

    try:
        df = pd.read_csv("data/all_reviews.csv", sep=';', encoding="Windows-1251")
        sentiments = ["negative"]
        if process_neutral:
            sentiments.append("neutral")
        comments = df.loc[df["sentiment"].isin(sentiments), ["text", "sentiment"]].copy()
        comments.rename(columns={"text": "comment", "sentiment": "tone"}, inplace=True)
        comments["comment"] = comments["comment"].astype(str)
    except FileNotFoundError:
        print("Error: data/all_reviews.csv not found. Using sample data.")
        comments = pd.DataFrame([
            {"comment": "Однажды внезапно заблокировали все аккаунты", "tone": "negative"},
            {"comment": "Приложение часто вылетает", "tone": "negative"},
            {"comment": "Поддержка отвечает нормально", "tone": "neutral"}
        ])
        comments = comments[comments["tone"].isin(sentiments)]

    comments["aspect"] = comments["comment"].apply(extract_aspect)
    comments["recommendations"] = comments["aspect"].apply(get_prewritten_recommendations)

    comments["is_valid"] = comments["recommendations"].apply(is_valid_recommendation)
    valid_comments = comments[comments["is_valid"]].drop(columns=["is_valid"])
    invalid_count = len(comments) - len(valid_comments)
    if invalid_count > 0:
        print(f"Warning: {invalid_count} invalid recommendations filtered out.")

    csv_path = "data/recommendations.csv"
    valid_comments.to_csv(csv_path, index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)
    print(f"Saved {len(valid_comments)} recommendations to {csv_path}")

    embedder = SentenceTransformer(embedder_model)
    valid_comments["embedding"] = valid_comments["comment"].apply(lambda x: pickle.dumps(embedder.encode(x)))

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    for _, row in valid_comments.iterrows():
        cursor.execute("""
            INSERT OR REPLACE INTO feedback (comment, tone, aspect, recommendations, embedding)
            VALUES (?, ?, ?, ?, ?)
        """, (row["comment"], row["tone"], row["aspect"], row["recommendations"], row["embedding"]))
    conn.commit()
    conn.close()
    print(f"Inserted {len(valid_comments)} records into {db_path}")

populate_db()
print("\nCheck recommendations.csv:")
print(pd.read_csv("data/recommendations.csv", encoding="utf-8"))
print("\nCheck feedback.db:")
conn = sqlite3.connect("data/feedback.db")
cursor = conn.cursor()
cursor.execute("SELECT comment, tone, aspect, recommendations FROM feedback")
results = cursor.fetchall()
print(f"Found {len(results)} records:")
for row in results[:5]:  # Limit to 5
    print(row)
conn.close()

Saved 77 recommendations to data/recommendations.csv
Inserted 77 records into data/feedback.db

Check recommendations.csv:
                                              comment      tone   aspect  \
0   Однажды внезапно заблокировали все аккаунты на...  negative  general   
1   Есть все, что нужно, но дружелюбность интерфей...  negative  general   
2   Случайно заметил расхождение на 10% не в мою п...  negative  general   
3   Были здесь на профориентационной экскурсии для...  negative  general   
4   Отвратительное отношение к клиентам. Оценка на...  negative  general   
..                                                ...       ...      ...   
72  В личном кабинете сделали заявку на разблокиро...  negative  general   
73  отвратительно,категорический не рекомендую ком...  negative  general   
74  Остались смешанные впечатления… До операторов ...  negative  general   
75  все устраивает. и денег водителям в дорогу дае...  negative  general   
76  Отвратительная компания, не возможно 

In [5]:
class Retriever:
    def __init__(self):
        config = load_config()
        self.db_path = config["database"]["path"]
        self.model = SentenceTransformer(config["models"]["embedder"])
        self.top_k = config["rag"]["top_k"]
        self.threshold = config["rag"]["similarity_threshold"]

    def retrieve(self, query):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT comment, recommendations, embedding FROM feedback WHERE embedding IS NOT NULL")
        rows = cursor.fetchall()
        if not rows:
            print("No records found in feedback.db")
            return []
        comments = [row[0] for row in rows]
        recommendations = [row[1] or "" for row in rows]
        embeddings = np.array([pickle.loads(row[2]) for row in rows])
        conn.close()
        query_embedding = self.model.encode(query)
        similarities = cosine_similarity([query_embedding], embeddings)[0]
        top_indices = np.argsort(similarities)[-self.top_k:][::-1]
        results = [
            (comments[i], recommendations[i], float(similarities[i]))
            for i in top_indices if similarities[i] >= self.threshold
        ]
        if not results and self.threshold > 0.5:
            results = [
                (comments[i], recommendations[i], float(similarities[i]))
                for i in top_indices if similarities[i] >= 0.5
            ]
        return results

retriever = Retriever()
results = retriever.retrieve("Задержка возврата денег")
print("Retriever results (top 5):")
for result in results[:5]:
    print(f"Comment: {result[0][:50]}...\nRecommendation: {result[1]}\nSimilarity: {result[2]:.3f}\n")

Retriever results (top 5):
Comment: После подписания документов на расторжение договор...
Recommendation: 1. Improve overall customer communication.
2. Gather regular feedback through surveys.
3. Enhance user interface for better experience.
4. Offer loyalty discounts to retain customers.
Similarity: 0.709

Comment: После подписания документов на расторжение договор...
Recommendation: 1. Improve overall customer communication.
2. Gather regular feedback through surveys.
3. Enhance user interface for better experience.
4. Offer loyalty discounts to retain customers.
Similarity: 0.709

Comment: Ничего не понравилось. Наоборот закрыл договор и о...
Recommendation: 1. Improve overall customer communication.
2. Gather regular feedback through surveys.
3. Enhance user interface for better experience.
4. Offer loyalty discounts to retain customers.
Similarity: 0.650

Comment: Ничего не понравилось. Наоборот закрыл договор и о...
Recommendation: 1. Improve overall customer communication.
2. Ga

In [6]:
def mock_recommender(comments):
    df = pd.DataFrame({"comment": comments})
    df["aspect"] = df["comment"].apply(extract_aspect)
    df["recommendations"] = df["aspect"].apply(get_prewritten_recommendations)
    return df["recommendations"].tolist()

def mock_pipeline():
    config = load_config()
    db_path = config["database"]["path"]
    process_neutral = config.get("processing", {}).get("include_neutral", False)

    try:
        df = pd.read_csv("data/all_reviews.csv", sep=';', encoding="windows-1251")
        sentiments = ["negative"]
        if process_neutral:
            sentiments.append("neutral")
        comments = df.loc[df["sentiment"].isin(sentiments), ["text"]].copy()
        comments["text"] = comments["text"].astype(str)
        comment_list = comments["text"].tolist()[:10]  # Limit for testing
    except FileNotFoundError:
        print("Error: data/all_reviews.csv not found. Exiting.")
        return

    recommendations = mock_recommender(comment_list)

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    updated = 0
    for comment, rec in zip(comment_list, recommendations):
        cursor.execute("UPDATE feedback SET recommendations = ? WHERE comment = ?", (rec, comment))
        if cursor.rowcount > 0:
            updated += 1
        print(f"Comment: {comment[:50]}...\nRecommendation: {rec}\n")
    conn.commit()
    conn.close()
    print(f"Pipeline completed, updated {updated} records")

mock_pipeline()
print("\nCheck updated feedback.db:")
conn = sqlite3.connect("data/feedback.db")
cursor = conn.cursor()
cursor.execute("SELECT comment, tone, aspect, recommendations FROM feedback")
results = cursor.fetchall()
print(f"Found {len(results)} records:")
for row in results[:5]:
    print(row)
conn.close()

Comment: Однажды внезапно заблокировали все аккаунты нашей ...
Recommendation: 1. Improve overall customer communication.
2. Gather regular feedback through surveys.
3. Enhance user interface for better experience.
4. Offer loyalty discounts to retain customers.

Comment: Есть все, что нужно, но дружелюбность интерфейса о...
Recommendation: 1. Improve overall customer communication.
2. Gather regular feedback through surveys.
3. Enhance user interface for better experience.
4. Offer loyalty discounts to retain customers.

Comment: Случайно заметил расхождение на 10% не в мою польз...
Recommendation: 1. Improve overall customer communication.
2. Gather regular feedback through surveys.
3. Enhance user interface for better experience.
4. Offer loyalty discounts to retain customers.

Comment: Были здесь на профориентационной экскурсии для шко...
Recommendation: 1. Improve overall customer communication.
2. Gather regular feedback through surveys.
3. Enhance user interface for better exper